In [1]:
import os
import librosa
import moviepy.editor as mp
import numpy as np
from sklearn.ensemble import RandomForestClassifier

In [2]:
def extract_features(audio_path):
    y, sr = librosa.load(audio_path)

    # Example feature extraction - customize based on your data
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    mfccs = librosa.feature.mfcc(y=y, sr=sr)
    spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
    rms = librosa.feature.rms(y=y)[0]

    # Ensure all feature arrays have the same number of columns
    min_len = min(chroma_stft.shape[1], mfccs.shape[1], spectral_contrast.shape[1], rms.shape[0])
    chroma_stft = chroma_stft[:, :min_len]
    mfccs = mfccs[:, :min_len]
    spectral_contrast = spectral_contrast[:, :min_len]
    rms = rms[:min_len]

    return chroma_stft, mfccs, spectral_contrast, rms

In [3]:
def train_classifier(features, labels):
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(features, labels)
    return clf

In [4]:
def detect_bat_hit(classifier, audio_features):
    predictions = classifier.predict(audio_features)
    return predictions

In [5]:
def initiate_process():
    # Replace 'positive_samples' and 'negative_samples' with your 
    # actual directories containing positive and negative samples
    positive_samples_dir = '../Media/positive_samples'
    negative_samples_dir = '../Media/negative_samples'

    # Read video files from a directory
    video_directory = '../Media/videos_to_be_processed'
    video_files = [f for f in os.listdir(video_directory) if f.endswith('.mp4')]
    
    #----------------------------------------------------
    
    # Extract features for positive samples (bat hits)
    positive_features = []
    positive_labels = []
    for positive_audio_file in os.listdir(positive_samples_dir):
        positive_audio_path = os.path.join(positive_samples_dir, positive_audio_file)
        features = extract_features(positive_audio_path)
        for feat in features:
            flattened_feat = feat.flatten()
            #print(f"Shape before flattening: {feat.shape}, Shape after flattening: {flattened_feat.shape}")
            positive_features.append(flattened_feat)
            positive_labels.append(1)  # Positive label for bat hit

    # Extract features for negative samples (non-bat hits)
    negative_features = []
    negative_labels = []
    for negative_audio_file in os.listdir(negative_samples_dir):
        negative_audio_path = os.path.join(negative_samples_dir, negative_audio_file)
        features = extract_features(negative_audio_path)
        for feat in features:
            flattened_feat = feat.flatten()
            #print(f"Shape before flattening: {feat.shape}, Shape after flattening: {flattened_feat.shape}")
            negative_features.append(flattened_feat)
            negative_labels.append(0)  # Negative label for non-bat hit

    # Ensure all feature arrays have the same number of columns
    min_len = min(len(positive_features), len(negative_features))
    positive_features = positive_features[:min_len]
    negative_features = negative_features[:min_len]

    # Combine positive and negative samples
    all_features = np.concatenate([positive_features, negative_features], axis=0)
    all_labels = np.concatenate([positive_labels, negative_labels])

    # Ensure all feature arrays have the same number of columns
    min_len = min(len(positive_features), len(negative_features))
    all_features = all_features[:min_len]
    all_labels = all_labels[:min_len]
    
    #----------------------------------------------------
    
    # Train the classifier
    classifier = train_classifier(all_features, all_labels.reshape(-1))

    # Test the classifier on video files from the directory
    for video_file in video_files:
        video_path = os.path.join(video_directory, video_file)
        audio_path = 'temp_audio.wav'

        # Extract audio from video
        video_clip = mp.VideoFileClip(video_path)
        video_clip.audio.write_audiofile(audio_path, fps=video_clip.fps)

        # Extract features for the test video
        test_chroma_stft, test_mfccs, test_spectral_contrast, test_rms = extract_features(audio_path)
        test_features = np.vstack((test_chroma_stft, test_mfccs, test_spectral_contrast, test_rms)).T

        # Detect bat hits using the trained classifier
        predictions = detect_bat_hit(classifier, test_features)

        # Check if there are any positive predictions indicating a bat hit
        if 1 in predictions:
            print(f"Cricket ball hit the bat in {video_file}")
        else:
            print(f"No cricket ball hit the bat in {video_file}")

In [6]:
initiate_process()

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (12,) + inhomogeneous part.